<a href="https://colab.research.google.com/github/junghodavidlee/Project-3/blob/maddie_jankowski/Project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
! pip install fastparquet
!pip install gradio
!pip install vaderSentiment
!pip install nltk
!pip install datasets

In [25]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt
from datasets import load_dataset
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
import gradio as gr
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import joblib
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.svm import LinearSVC


# Set the column width to view the text message data.
pd.set_option('max_colwidth', 200)

In [2]:
# Import the IMDB dataset
from google.colab import files
uploaded = files.upload()


Saving IMDB Dataset.csv to IMDB Dataset (2).csv


In [3]:
# Import the IMDB dataset
imdb_df = pd.read_csv('IMDB Dataset.csv')
imdb_df.head()


,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me...",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire p...",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue i...",positive
3,Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenl...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what mone...",positive


In [4]:
# Change the name of the review column to "text" and the sentiment column to "label"
imdb_df.rename(columns={'review': 'text', 'sentiment': 'label'}, inplace=True)
imdb_df.head()

,text,label
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me...",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire p...",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue i...",positive
3,Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenl...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what mone...",positive


In [5]:
# Change the label column to have positive be 1 and negative be 0
imdb_df['label'] = imdb_df['label'].map({'positive': 1, 'negative': 0})
imdb_df.head()

,text,label
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me...",1
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire p...",1
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue i...",1
3,Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenl...,0
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what mone...",1


In [6]:
# Set the features variable to the "text" column
X = imdb_df['text']
# Set the target variable to the "label" column
y = imdb_df['label']
# Split the data into train, test, split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
custom_stopwords = ['a', 'about', 'an', 'and', 'are', 'as', 'at', 'be', 'been', 'but', 'by', 'can', \
             'even', 'ever', 'for', 'from', 'get', 'had', 'has', 'have', 'he', 'her', 'hers', 'his', \
             'how', 'i', 'if', 'in', 'into', 'is', 'it', 'its', 'just', 'me', 'my', 'of', 'on', 'or', \
             'see', 'seen', 'she', 'so', 'than', 'that', 'the', 'their', 'there', 'they', 'this', \
             'to', 'was', 'we', 'were', 'what', 'when', 'which', 'who', 'will', 'with', 'you']

In [26]:
# Build a pipeline using TfidfVectorizer(), with custom_stopwords and LinearSVC
text_clf = Pipeline([('tfidf', TfidfVectorizer(stop_words=custom_stopwords, ngram_range=(1,2), max_df=0.9, min_df=2, sublinear_tf=True)),
                     ('clf', LinearSVC(C=1, loss='squared_hinge', penalty='l2', dual=True))])

# Fit the data to the model
text_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.9, min_df=2, ngram_range=(1, 2),
                                 stop_words=['a', 'about', 'an', 'and', 'are',
                                             'as', 'at', 'be', 'been', 'but',
                                             'by', 'can', 'even', 'ever', 'for',
                                             'from', 'get', 'had', 'has',
                                             'have', 'he', 'her', 'hers', 'his',
                                             'how', 'i', 'if', 'in', 'into',
                                             'is', ...],
                                 sublinear_tf=True)),
                ('clf', LinearSVC(C=1, dual=True))])

In [29]:
# Build a pipeline using TfidfVectorizer(), with custom_stopwords and LinearSVC
text_clf2 = Pipeline([('tfidf', TfidfVectorizer(stop_words=custom_stopwords, ngram_range=(1,2), max_df=0.9, min_df=2, sublinear_tf=True)),
                     ('clf', MultinomialNB())])

# Fit the data to the model
# Convert the sparse matrix output of TfidfVectorizer to a dense array
text_clf2.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.9, min_df=2, ngram_range=(1, 2),
                                 stop_words=['a', 'about', 'an', 'and', 'are',
                                             'as', 'at', 'be', 'been', 'but',
                                             'by', 'can', 'even', 'ever', 'for',
                                             'from', 'get', 'had', 'has',
                                             'have', 'he', 'her', 'hers', 'his',
                                             'how', 'i', 'if', 'in', 'into',
                                             'is', ...],
                                 sublinear_tf=True)),
                ('clf', MultinomialNB())])

In [30]:
# Validate the model by checking the model accuracy with model.score
print('Train accuracy:', text_clf.score(X_train, y_train))
print('Test accuracy:', text_clf.score(X_test, y_test))

Train accuracy: 0.9998
Test accuracy: 0.9171


In [31]:
# Validate the model by checking the model accuracy with model.score
print('Train accuracy:', text_clf2.score(X_train, y_train))
print('Test accuracy:', text_clf2.score(X_test, y_test))

Train accuracy: 0.961275
Test accuracy: 0.8938


In [32]:
# Create a confusion matrix on the test data and predictions
predictions = text_clf.predict(X_test)
print(metrics.confusion_matrix(y_test,predictions))
print(metrics.classification_report(y_test,predictions))
print(metrics.accuracy_score(y_test,predictions))

[[4508  453]
 [ 376 4663]]
              precision    recall  f1-score   support

           0       0.92      0.91      0.92      4961
           1       0.91      0.93      0.92      5039

    accuracy                           0.92     10000
   macro avg       0.92      0.92      0.92     10000
weighted avg       0.92      0.92      0.92     10000

0.9171


In [33]:
# Create a confusion matrix on the test data and predictions
predictions = text_clf2.predict(X_test)
print(metrics.confusion_matrix(y_test,predictions))
print(metrics.classification_report(y_test,predictions))
print(metrics.accuracy_score(y_test,predictions))

[[4497  464]
 [ 598 4441]]
              precision    recall  f1-score   support

           0       0.88      0.91      0.89      4961
           1       0.91      0.88      0.89      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000

0.8938


In [34]:
# Test a sample review "I thought the movie was slow and I wasn't very engaged"
sample = text_clf.predict(["I thought the movie was slow and I wasn't very engaged"])
sample

array([0])

In [37]:
# Test a sample review "I thought the movie was slow and I wasn't very engaged"
sample = text_clf2.predict(["I thought the movie was slow and I wasn't very engaged"])
sample

array([0])

In [35]:
# Test another sample review "This movie brought me so much joy. I loved the part with the animals"
sample = text_clf.predict(["This movie brought me so much joy. I loved the part with the animals"])
sample

array([1])

In [38]:
# Test another sample review "This movie brought me so much joy. I loved the part with the animals"
sample = text_clf2.predict(["This movie brought me so much joy. I loved the part with the animals"])
sample

array([1])

In [39]:
# Save the model
vectorizer = text_clf.named_steps['tfidf']
joblib.dump(vectorizer, 'vectorizer.pkl')
model = text_clf.named_steps['clf']
joblib.dump(model, 'model.pkl')

['model.pkl']

In [40]:
#Load the saved model
new_vectorizer = joblib.load('vectorizer.pkl')
new_model = joblib.load('model.pkl')

In [41]:
# Load the rt dataset
rt = load_dataset("cornell-movie-review-data/rotten_tomatoes")

In [42]:
# Extract the train dataset from the rt
rt_df = rt['train'].to_pandas()

In [43]:
rt_df.head()

,text,label
0,"the rock is destined to be the 21st century's new "" conan "" and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .",1
1,"the gorgeously elaborate continuation of "" the lord of the rings "" trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r ...",1
2,effective but too-tepid biopic,1
3,"if you sometimes like to go to the movies to have fun , wasabi is a good place to start .",1
4,"emerges as something rare , an issue movie that's so honest and keenly observed that it doesn't feel like one .",1


In [44]:
rt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8530 entries, 0 to 8529
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    8530 non-null   object
 1   label   8530 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 133.4+ KB


In [45]:
# Assuming the text column is named 'review'
new_texts = rt_df['text'].astype(str)  # Convert to string if necessary

# Transform using the existing vectorizer
new_features = new_vectorizer.transform(new_texts)

In [46]:
from sklearn.metrics import accuracy_score, classification_report

# Get actual labels
y_true = rt_df['label']

# Predict using the trained model
y_pred = new_model.predict(new_features)

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Detailed evaluation
print(classification_report(y_true, y_pred))


Accuracy: 0.7462
              precision    recall  f1-score   support

           0       0.77      0.70      0.73      4265
           1       0.72      0.80      0.76      4265

    accuracy                           0.75      8530
   macro avg       0.75      0.75      0.75      8530
weighted avg       0.75      0.75      0.75      8530



In [47]:

# Load your vectorizer and transform the dataset
vectorizer = joblib.load('vectorizer.pkl')
X = vectorizer.transform(rt_df['text'])  # Text features
y = rt_df['label']  # Target labels

# Initialize MultinomialNB
model = MultinomialNB()

# Perform 5-fold cross-validation (you can adjust the number of folds)
cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

# Output the accuracy scores for each fold
print(f"Cross-validation scores: {cv_scores}")
print(f"Mean accuracy: {cv_scores.mean()} ± {cv_scores.std()}")


Cross-validation scores: [0.76143025 0.77784291 0.7919109  0.78077374 0.77256741]
Mean accuracy: 0.7769050410316529 ± 0.009993063925498744


In [48]:
#Download Vader package
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
# gradio interface low quality
func = gr.Interface(
    fn=vader_review,
    inputs=gr.Textbox(
    label="Enter your review here",
    lines=3,
    max_lines=5,
    interactive=True
),
    outputs=gr.Textbox(label='Review')
)
func.launch()

In [66]:
# Better gradio interface


# Sentiment Analysis Function returning sentiment and score
def vader_review(text):
    analyzer = SentimentIntensityAnalyzer()
    score = analyzer.polarity_scores(text)['compound']
    if score >= 0.05:
        return "😊 Positive", score
    elif score <= -0.05:
        return "😠 Negative", score
    else:
        return "😐 Neutral", score

# Improved Gradio Interface
with gr.Blocks(css=".gradio-container {background-color: #f5f7fa; font-family: Arial;}") as demo:
    gr.Markdown("## 🌟 Sentiment Analyzer 🌟")
    gr.Markdown("Enter your review, and we'll analyze its sentiment!")

    with gr.Row():
        input_text = gr.Textbox(
            label="📝 Enter your review",
            lines=3,
            max_lines=5,
            interactive=True,
            placeholder="Type your review here..."
        )

    analyze_button = gr.Button("🔍 Analyze Sentiment")

    with gr.Row():
        output_label = gr.Textbox(label="🔽 Sentiment Result", interactive=False)

    analyze_button.click(vader_review, input_text, output_label)

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://577e5dde7346f8f2eb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [67]:
# Sentiment Analysis Function returning sentiment and score using text_clf
def mj_review(text):
    score = text_clf.predict([text])[0]
    if score == 1:
        return "😊 Positive"
    elif score == 0:
        return "😠 Negative"

# Improved Gradio Interface
with gr.Blocks(css=".gradio-container {background-color: #f5f7fa; font-family: Arial;}") as demo:
    gr.Markdown("## 🌟 Sentiment Analyzer 🌟")
    gr.Markdown("Enter your review, and we'll analyze its sentiment!")

    with gr.Row():
        input_text = gr.Textbox(
            label="📝 Enter your review",
            lines=3,
            max_lines=5,
            interactive=True,
            placeholder="Type your review here..."
        )

    analyze_button = gr.Button("🔍 Analyze Sentiment")

    with gr.Row():
        output_label = gr.Textbox(label="🔽 Sentiment Result", interactive=False)

    analyze_button.click(mj_review, input_text, output_label)

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5f5cfcd14c4465b5a6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
